In [7]:
import geopandas as gpd
import numpy as np
import pandas as pd
import pyarrow

import pysal as ps
import matplotlib.pyplot as plt
import matplotlib.collections as mplc
%matplotlib inline
import seaborn as sns
import shapely

from shapely import geometry as sgeom
from shapely.geometry import Point
from shapely import ops

import warnings
warnings.simplefilter('ignore') 

In [11]:
consco_path = 'connectivity_scores_EW.parquet'


con_data = pd.read_parquet('../' + consco_path, engine='pyarrow')

In [12]:
con_data

,lsoa,Education_car,Business_car,Shopping_car,Education_walk,Business_pt,Education_pt,Visit friends at private home_car,Entertain / public activity_car,Business_walk,...,overall_decile,overall_walk_decile,overall_car_decile,overall_cycling_decile,overall_pt_decile,overall_business_decile,overall_education_decile,overall_entertain_decile,overall_shopping_decile,overall_visit_decile
0,E01005863,92.573614,88.640400,87.550186,73.153322,85.000465,87.710114,95.136887,86.912791,72.805128,...,9,7,9,8,9,9,8,9,8,9
1,E01017132,85.265883,80.949751,78.407571,88.657543,75.177727,81.726362,89.988320,76.683146,75.204019,...,7,9,5,7,8,8,8,9,3,8
2,W01000112,57.162763,58.237645,53.278282,18.443314,40.212661,13.578862,68.390217,53.290994,35.679213,...,0,0,0,0,0,0,0,0,0,0
3,E01001819,96.615112,95.825290,92.092425,93.338159,91.982896,93.075689,98.362011,92.628798,84.415298,...,9,9,9,9,9,9,9,9,9,9
4,E01011366,93.993131,89.093714,88.368712,70.077110,80.901211,66.759753,95.266358,85.036759,81.355135,...,8,7,9,8,6,9,7,8,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34748,E01032968,80.984612,79.402729,82.619903,23.391501,40.872350,22.259835,84.614027,82.060974,41.828958,...,1,0,4,1,0,0,1,1,0,0
34749,E01022973,82.758682,80.996364,80.506434,59.849031,65.237215,59.162966,89.901367,73.307320,66.520570,...,4,5,4,4,4,5,4,2,4,6
34750,E01017032,85.436225,81.079711,78.734144,89.038890,78.767808,83.659782,90.151978,76.502208,75.834969,...,7,9,5,7,9,8,8,9,5,8
34751,E01011303,83.048692,82.584986,78.015826,33.762679,56.141676,42.482136,88.164161,73.699489,50.744293,...,2,1,4,1,1,2,2,1,2,2
